In [2]:
import numpy as np
import pickle
import csv
import torch
from torch.linalg import inv, eig, pinv
from matplotlib import pyplot as plt
from tools import whiten
from sklearn import svm, metrics
from sklearn.decomposition import PCA
from datetime import datetime

In [3]:
def load(filename, reduced=True):
    sensor_data = []
    times = []
    responding_sens = [0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0]
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        # times = [row[0] for row in reader]
        for row in reader:
            if row[0] =='Timestamp':
                continue
            else:
                times.append(row[0])
                values = []
                for i in range(17):
                    b1 = int(row[2*i+1])
                    b2 = int(row[2*i+2])
                    values.append(int.from_bytes([b1, b2], byteorder="little"))
                sensor_data.append(values)
    sensor_data = np.array(sensor_data)
    if reduced:
        sensor_data = np.delete(sensor_data, np.where(np.array(responding_sens)==0)[0], axis=1)
    sequence = pickle.load(open('data/1_300_20_sequence.pkl', 'rb'))
    # Convert to seconds
    times_sec = []
    for dt_str in times:
        dt = datetime.strptime(dt_str, '%Y-%m-%d %H:%M:%S.%f')
        seconds = dt.hour * 3600 + dt.minute * 60 + dt.second + dt.microsecond / 1e6
        times_sec.append(seconds)
    sequence_sec = []
    for dt_str in sequence:
        dt = datetime.strptime(dt_str[0], '%a %b %d %H:%M:%S %Y')
        seconds = dt.hour * 3600 + dt.minute * 60 + dt.second
        sequence_sec.append(seconds)
    times_sec = np.array(times_sec)
    sequence_sec = np.array(sequence_sec)
    return sensor_data, sequence, times_sec, sequence_sec

In [4]:
filename = 'data/1_300_20.csv'

sensor_data, sequence, times_sec, sequence_sec = load(filename, reduced=False)

In [5]:
delay = 1.5
t_baseline = 300
n_train = 225


baseline = np.mean(sensor_data[:t_baseline], axis=0)
X_train = []
Y_train = []
I_train = []
X_test = []
Y_test = []
I_test = []
counts = np.zeros((3))

for i, t in enumerate(sequence_sec):
    try:
        flags = (times_sec > sequence_sec[i]) & (times_sec < sequence_sec[i+1] + delay)
    except IndexError:
        flags = (times_sec > sequence_sec[i])
    sample = sensor_data[flags][:18]
    t_sample = times_sec[flags][:18]

    responding_sens = [0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0]
    sample = np.delete(sample, np.where(np.array(responding_sens)==0)[0], axis=1)

    x0 = sample[0, :]
    t0 = t_sample[0]
    sum = np.zeros((sample.shape[1]))
    for k in range(1, sample.shape[0]):
        dif = np.abs(sample[k, :] - x0)/(t_sample[k] - t0)
        sum += dif

    if counts[sequence[i][1]-1] < n_train//3:
        X_train.append(sample.flatten())
        Y_train.append(sequence[i][1]-1)
        I_train.append(sum)
        counts[sequence[i][1]-1] += 1
    else:
        X_test.append(sample.flatten())
        Y_test.append(sequence[i][1]-1)
        I_test.append(sum)

    # if i<225:
    #     X_train.append(sample.flatten())
    #     Y_train.append(sequence[i][1]-1)
    # else:
    #     X_test.append(sample.flatten())
    #     Y_test.append(sequence[i][1]-1)

X_train = np.array(X_train)
Y_train = np.array(Y_train)
I_train = np.array(I_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)
I_test = np.array(I_test)

In [7]:
n_out = 3
n_hd = 10000
k = 50
w_teacher = 1.
n_pot = 10
uniformW = False
train_epoch = 20
n_dense = I_train.shape[1]

if uniformW:
    W_hd = np.random.uniform(high=1/np.sqrt(n_dense), size=(n_hd, n_dense))  #Test random sparse weights
else:
    W_hd = np.random.binomial(n=1, p=0.05, size=(n_hd, n_dense))  #Test random sparse weights

W_out = np.zeros((n_out, n_hd))
W = np.zeros((n_out, n_hd))

for epoch in range(train_epoch):
    x_dense = I_train
    labels = np.concatenate((Y_train, Y_test))

    x_hd = x_dense @ W_hd.T
    z_hd = np.where(np.argsort(x_hd)<k, 1., 0)

    z_out_train = np.zeros((z_hd.shape[0],  n_out))
    for i, row in enumerate(z_hd):
        teacher = np.zeros((n_out,))
        if labels[i] != 0:
            teacher[int(labels[i])] = w_teacher
        out = row @ W_out.T + teacher
        z_out_train[i] = out
        dW = (1./n_pot)*(np.atleast_2d(out).T @ np.atleast_2d(row))
        W += dW
        W_out = np.where(W>=1., 1./k, 0.)
        # if i%100 == 0:
        #     print(np.sum(W_out, axis=1))

    x_dense = np.vstack((I_train, I_test))
    x_hd = x_dense @ W_hd.T
    z_hd = np.where(np.argsort(x_hd)<k, 1., 0)

    z_out = np.zeros((z_hd.shape[0],  n_out))
    for i, row in enumerate(z_hd):
        out = row @ W_out.T
        z_out[i] = out

    # z_wta = np.where(np.argsort(z_out, axis=1)<1, 1., 0)

    z_pred = np.zeros_like(z_out)
    z_pred = np.argsort(z_out, axis=1)[:, -1]

    train_acc = metrics.accuracy_score(labels[:n_train], z_pred[:n_train])
    test_acc = metrics.accuracy_score(labels[n_train:], z_pred[n_train:])

    # results['train_acc'].append(train_acc)
    # results['test_acc'].append(test_acc)
    #
    # print(f'UniformW: {uniformW}, Normalized: {normalized}, Whitened: {whitened}, k: {k}, n_pot: {n_pot}, t_training_delay: {t_training_delay}, n_fold: {n_fold}')
    print(f'Epoch: {epoch}\tTrain accuracy: {train_acc:.4f}\tTest accuracy: {test_acc:.4f}')


Epoch: 0	Train accuracy: 0.3822	Test accuracy: 0.3600
Epoch: 1	Train accuracy: 0.5067	Test accuracy: 0.4533
Epoch: 2	Train accuracy: 0.5778	Test accuracy: 0.4533
Epoch: 3	Train accuracy: 0.6400	Test accuracy: 0.4800
Epoch: 4	Train accuracy: 0.6356	Test accuracy: 0.4533
Epoch: 5	Train accuracy: 0.6444	Test accuracy: 0.4400
Epoch: 6	Train accuracy: 0.6622	Test accuracy: 0.4267
Epoch: 7	Train accuracy: 0.6533	Test accuracy: 0.4267
Epoch: 8	Train accuracy: 0.6533	Test accuracy: 0.4267
Epoch: 9	Train accuracy: 0.6533	Test accuracy: 0.4267
Epoch: 10	Train accuracy: 0.6533	Test accuracy: 0.4267
Epoch: 11	Train accuracy: 0.6222	Test accuracy: 0.4000
Epoch: 12	Train accuracy: 0.5600	Test accuracy: 0.3867
Epoch: 13	Train accuracy: 0.4711	Test accuracy: 0.4000
Epoch: 14	Train accuracy: 0.4311	Test accuracy: 0.3733
Epoch: 15	Train accuracy: 0.4089	Test accuracy: 0.3867
Epoch: 16	Train accuracy: 0.3778	Test accuracy: 0.3600
Epoch: 17	Train accuracy: 0.3689	Test accuracy: 0.3600
Epoch: 18	Train accu